# INF0413 - Processamento Digital de Sinais e Imagens
A02 - Segunda avaliação da aprendizagem

**Lecturer / Examiner**:

Aldo Díaz (aldo.diaz@ufg.br)

**Teaching Assistants:**

- Gabriel Van Der Schmidt (gabriel_schmidt@discente.ufg.br)

- Lucca Emmanuel Pineli Simões (lucca.pineli@discente.ufg.br)

- Lucas Simões Passos (lucas.simoes@discente.ufg.br)

## Introdução

A classificação de sinais acústicos é uma capacidade cognitiva que nós, enquanto humanos, realizamos de forma *natural* e em todo instante de tempo (será que alguem pode escolher desligar ela?).
Noentanto, trata-se de uma *tarefa complexa* para as máquinas devido à grande diversidade e complexidade desses sinais ao serem tratados com técnicas de processamento de sinais como dados em arquivos de áudio.

O [Environmental Sound Classification 50 (ESC50)](https://www.kaggle.com/datasets/mmoreaux/environmental-sound-classification-50/) é um dataset anotado com 2000 amostras de som ambiente de várias fontes, mirado no uso como benchmark de métodos de classificação.

| | | | |
|:-------------------------:|:-------------------------:|:-------------------------:|:-------------------------:|
|<img height="250" width="250" alt="" src="https://yt3.googleusercontent.com/eY3K4zuKBcGxymSbuxwYHEn6vPt_Az3G9SUImKetmcGe8mbh75rbHGXg4xE8ojVJtatz7d0RPQw=s176-c-k-c0x00ffffff-no-rj"> | <img height="250" width="250" alt="" src="https://www.meloteca.com/wp-content/uploads/2018/11/tecnica-vocal-300x300.jpg"> | <img height="250" width="250" alt="" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/Chainsaw.JPG/220px-Chainsaw.JPG"> | <img height="250" width="250" alt="" src="https://aviationvoice.com/wp-content/uploads/2017/03/Increasing-your-helicopters%E2%80%99-efficiency-mission-impossible.jpg">

É composto por 2000 clips de 5 segundos cada, uniformemente distribuídas ao longo de 50 classes incluindo sons de origem humana, doméstica e natural. Para simplicar o seu trabalho, utilizaremos um subset (ESC-10) contendo apenas 10 classes, 400 amostras ao todo.

In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy.io import wavfile
import matplotlib.pyplot as plt
import seaborn as sns
import os

**Baixe e confira o dataset antes de começar a prova!**

In [ ]:
# Instalar biblioteca auxiliar
!pip install gdown keras tensorflow visualkeras

In [ ]:
# Baixar os arquivos
!gdown 1LtZf0J7mUzDjIeT57IcpqtVQmzqAMgYF
!gzip -dc 'esc50.tar.gz' | tar xvf -

In [ ]:
root = '.' # '/content'
esc50 = pd.read_csv(os.path.join(root, 'esc50.csv'))

print(esc50)

In [ ]:
# Selecionar apenas as amostras que pertencem ao dataset ESC10
esc10 = esc50.query('esc10 == True')

print(esc10)

In [ ]:
files = np.array(esc10['filename'])
y     = np.array(esc10['target'])

print(files[:20])
print(y[:20])

In [ ]:
# Carregar arquivos de áudio
# Conferir que todos tenham 5 segundos de duração e fs=16kHz
fs = 16000
X = []
k = 1
for file in files:
    filename = os.path.join(root, 'esc50', file)
    if not os.path.isfile(filename):
        print(f'ERROR: File {filename} not found!')
        break
    fs_tmp, signal = wavfile.read(filename)
    print(f'{k} - Read file {file}.')
    k += 1
    if fs != fs_tmp:
        print(f'ERROR: File {filename} has fs different than expected. Expected {fs}, got {fs_tmp}!')
        break
    X.append(signal)

print(len(X))

In [ ]:
# Checar que todos os sinais tenham a mesma dimensão
signal_lens = [len(signal) for signal in X]

print(np.unique(signal_lens)) # Sim!

In [ ]:
# E então converter para um array numpy
X = np.array(X)

print(X.shape)

In [ ]:
# Dicas/Funções úteis - parte 1

# Obter STFT e plotar
def STFT(sinal, tamanho_janela=256, tamanho_passo=128):
    '''
        Calcula a STFT de um sinal.
        Parâmetros:
        - sinal: np.array
            O sinal de entrada.
        - tamanho_janela: int
            Tamanho da janela para cada segmento.
        - tamanho_passo: int
            Número de amostras entre frames sucessivos.
        Retorna:
        - matriz_stft: np.array
            Matriz contendo os coeficientes STFT.
    '''
    num_amostras = len(sinal)
    num_janelas  = 1 + (num_amostras - tamanho_janela) // tamanho_passo

    # Prepara a função de janela (janela de Hann neste caso)
    janela = np.hanning(tamanho_janela)

    # Inicializa a matriz STFT
    matriz_stft = np.zeros((num_janelas, tamanho_janela // 2 + 1), \
                           dtype=np.complex128)

    # Calcula a STFT
    for k in range(num_janelas):
        inicio = k * tamanho_passo
        fim = inicio + tamanho_janela
        segmento = sinal[inicio:fim]

        # Aplica a função de janela
        segmento_janelado = segmento * janela

        # Calcula a FFT
        resultado_fft = np.fft.fft(segmento_janelado)

        # Armazena apenas as frequências não negativas (até a frequência de Nyquist)
        matriz_stft[k, :] = resultado_fft[:tamanho_janela // 2 + 1]

    return abs(matriz_stft)


def plot_spectrogram(sinal, fs, tamanho_janela=256, tamanho_passo=128):
    # Calcular espectrograma
    spec_signal = STFT(sinal, \
                       tamanho_janela=tamanho_janela, \
                       tamanho_passo=tamanho_passo)

    # Calcular shape do plot
    stft_signal_shape = spec_signal.T.shape
    print(stft_signal_shape)

    # Determinar tempo do audio e frequência máxima
    audio_duration = len(sinal)/fs
    window_size = (stft_signal_shape[0] - 1)*2
    max_freq = np.fft.fftfreq(window_size, 1 / fs)[window_size// 2 - 1]

    # Determinar eixos de tempo e frequencia
    time_values = np.linspace(0, audio_duration, stft_signal_shape[1])
    frequency_values = np.linspace(0, max_freq, stft_signal_shape[0])

    # Criar mapa
    heatmap = sns.heatmap(spec_signal.T, cmap = 'viridis')

    # Inverter o mapa
    heatmap.invert_yaxis()

    # Adicione rótulos aos eixos
    plt.title('Espectrograma do Sinal')
    plt.ylabel('Frequência (Hz)')
    plt.xlabel('Tempo (s)')

    # Setar eixos
    plt.xticks(np.arange(0, stft_signal_shape[1], step=stft_signal_shape[1] // 8), \
               labels=np.around(time_values[::stft_signal_shape[1] // 8], decimals=2), \
               rotation=45)
    plt.yticks(np.arange(0, stft_signal_shape[0], step=stft_signal_shape[0] // 8), \
               labels=np.around(frequency_values[::stft_signal_shape[0] // 8], decimals=2))
    plt.show()

In [ ]:
# Dicas/Funções úteis - parte 2
# (não rode esta célula, apenas copie, cole e edite onde você precisar)

# Plotar FFT
fft = np.abs(np.fft.fft(signal))
plt.stem(fft, 'k')
plt.show()

## Questão 1: (1,5 pontos)

- Obtenha e visualize o espectrograma de duas amostras de classes diferentes.

- Compare o gráfico dos respectivos sinais de áudio originais com o gráfico do espectro (magnitude da DFT).

- Quais as vantagens ou desvantagens de usar um ou outro e por quê (áudio bruto vs DFT vs espectrograma)?

In [ ]:
# Seu código aqui, lembre-se de comentar seus resultados

## Questão 2: (1,5 pontos)

Varie os valores dos parâmetros `tamanho_janela` (*default=256*) e `tamanho_passo` (*default=128*) na função geradora da transformada de Fourier de tempo curto (STFT).

- Como eles alteram a imagem do espectrograma resultante?

- Quais os possíveis usos/vantagens/desvantagens de alterar esses valores?

**Dica 1:**
Plote os resultados e observe os shapes das saídas.
Procure espectrogramas com respostas mais vibrantes para as mudanças ficarem mais óbvias.

**Dica 2:** A primeira dimensão de `STFT` corresponde às frequências, e a segunda corresponde ao tempo.

In [ ]:
# Seu código aqui, lembre-se de comentar seus resultados

## Questão 3: (5,5 pontos)

- Transforme os sinais de áudio em espectrogramas com valores de `tamanho_janela` e `tamanho_passo` de sua escolha.
Aproveite as conclusões da questão anterior e os resultados obtidos pelo modelo para escolher esses valores.

- Aplique métodos de pré-processamento (*e.g.*, redução de dimensionalidade) conforme for razoável.
Use o desempenho do modelo como guia.

- Treine um ***modelo de Regressão Logística*** com hiperparâmetros *default*.
Use uma divisão de treino e teste conforme a célula de código a seguir.
Avalie o desempenho de sua abordagem.

- Tente refinar sua abordagem com base na avaliação do modelo original.
Quais os impactos das mudanças feitas?

**Dica 1:** Para métodos de pré-processamento ou redução de dimensionalidade, use `fit`/`fit_transform` no conjunto de treino, mas apenas `transform` no conjunto de teste!

**Dica 2:** Métodos de normalização ou padronização da biblioteca `sklearn` normalizam "*coluna-a-coluna*" (feature-a-feature).
Como cada espectrograma possui uma amplitude diferente dos demais, o melhor aqui é normalizar "*amostra-a-amostra*".

**Dica 3:** `LogisticRegression` espera um array 2D de entrada de dimensões (`n_samples`, `n_features`).
Mas, o dataset de espectrogramas será 3D (`n_samples`, `spec_height`, `spec_width`).
Nesse caso, faça um "*reshape*" do array.

**Dica 4:** Mantenha seus testes organizados e ordenados.

**Dica 5:** Comente suas decisões, achados e conclusões.

In [ ]:
# Gerar espectrograma para cada sinal e concatenar em um único array numpy
X_spec = ... # Seu código aqui

In [ ]:
# WARNING!: NÃO MUDE O CONTEÚDO DESTA CÉLULA

# Depois de gerar o dataset de espetrogramas, dividir em sets de treino e teste
from sklearn.model_selection import train_test_split

# O parametro `stratify` garante que as distribuições das classes nos sets de
# treino/teste serão o mais próximas possível da distribuição original do
# dataset
X_train, X_test, y_train, y_test = train_test_split(X_spec, \
                                                    y, \
                                                    test_size=0.25, \
                                                    random_state=42, \
                                                    stratify=y)

In [ ]:
# Pre-processamento...
# Sugestões: sklearn.preprocessing.StandardScaler,
#            sklearn.preprocessing.MinMaxScaler,
#            sklearn.decomposition.PCA,
#            etc ...

# Seu código aqui...

In [ ]:
# Definição do modelo
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)

In [ ]:
# Treino do modelo
model.fit(X_train, y_train)

y_hat = model.predict(X_test)

In [ ]:
# Avaliação do modelo
from sklearn.metrics import classification_report

report_dict = classification_report(y_test, y_hat, output_dict=True)

print(report_dict)

In [ ]:
# ...

## Convolução: Uma potente ferramenta no processamento de sinais

Já que estamos trabalhando com espectrogramas e, como agora sabemos, espectrogramas são imagens, vamos introduzir brevemente uma técnica de processamento de sinais e imagens:
a famosa ***convolução***!

Em termos simples, a convolução de um sinal $x$ por outro sinal $y$ (chamado de kernel, máscara, ou janela) é o processo de **deslizar $y$ ao longo de $x$** e, em cada posição, fazer a **soma ponderada** dos elementos sobrepostos de $x$ e $y$ (os valores de $y$ são usados como pesos dessa ponderação).
Este processo gera um novo sinal $z$ que é dito de convolução.

<img src="https://www.researchgate.net/profile/Vincenzo-Randazzo/publication/346433900/figure/fig2/AS:962790265217050@1606558495822/Example-of-1D-Convolution-Neural-Network.jpg" alt="drawing" width="300"/>

Na imagem acima:
- o primeiro vetor é o sinal $x$,
- os pesos $w_1$, $w_2$ e $w_3$ formam $y$,
- o vetor final é $z$, gerado pela convolução de $x$ e $y$.

Como os sinais que estudamos até agora são vetores 1D, podemos extrapolar e pensar em **imagens como sinais 2D**!
Isso significa que podemos generalizar algumas técnicas de PDS de sinais 1D para sinais 2D.

No caso da convolução, ao invés de deslizar um vetor 1D sobre um outro, podemos deslizar uma matriz 2D (uma imagem, em nosso caso, um espectrograma!) sobre uma segunda matriz 2D (um kernel de convolução).
De novo, o resultado depende do kernel que escolhermos, sendo ele utilisado para tarefas como importantes, tais como a detecção de bordas, remoção de ruídos, suavização da imagem, etc.

Convoluções possuem diversas aplicações pois, mudando o tamanho e os valores da máscara, podemos obter resultados de processamento diferentes (os quais estudaremos na próxima unidade da disciplina).

<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*1okwhewf5KCtIPaFib4XaA.gif" alt="drawing" width="300"/>

Esse é o GIF de um [artigo bem bacana](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) que representa esse processo de "*deslizar*" uma matriz sobre a outra durante a convolução.
Aqui a matriz azul é a imagem original, a zona sobreada é o kernel sobreposto, e a matriz verde (azul esverdeado?) é a imagem resultante do processo de convolução.

### A escolha do kernel

Mas então, **como** que encontramos um "*bom kernel*"?

- Força bruta _ad aeternum_?

- Copiar e colar de trabalhos existentes?

Quem dera houvesse um algoritmo que ***automaticamente*** escolhesse pesos ótimos para esse kernel de acordo com a nossa necessidade, não é?

Pois bem, apresentamos para vocês as ***Redes Neurais Convolucionais*** (Convolutional Neural Networks - CNNs)!

### CNN

Uma CNN é uma rede neural que realiza várias convoluções e otimiza os valores dos kernels seguindo o gradiente descendente (backpropagation).
Esta é uma técnica perfeita para o processamento de imagens, pois "*extraimos informações*" da imagem, mas "*mantendo a sua disposição espacial*"!

Não se preocupe em entender tudo sobre CNNs agora, teremos tempo para isso no futuro.
O que importa agora é brincar um pouco com essa nova ferramenta e ver o que conseguimos fazer com ela.

## Questão 4: (1,5 pontos)

Crie um novo dataset de espectrogramas, como o feito na questão 3, com parâmetros default.

Treine e avalie o desempenho da CNN abaixo.

O resultado foi melhor ou pior que a sua abordagem na questão 3? Por que você acha que esse foi o caso?

- **Dica 1:** Essa CNN toma como entrada imagens 2D.
Não passe arrays 1D para ela.

- **Dica 2:** Use os valores default de `tamanho_janela` e `tamanho_passo` (256 e 128, respectivamente) nos espectrogramas utilisados para o treino da CNN, pois você precisará mudar o tamanho da entrada da rede (`input_size`) caso você use outros valores.

- **Dica 3:** Use os mesmos valores dos argumentos de `train_test_split` usados acima.

- **Dica 4:** Se o treinamento da rede estiver demorando muito, usar um ambiente com GPU vai acelerar o treinamento.

In [ ]:
import keras
from keras import Model
from keras.optimizers import Adam
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D
import visualkeras
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Gerar novo dataset de espectrogramas
X_spec = ... # Seu código aqui

# Para a CNN, `y` deve ser one-hot encoded
encoder = OneHotEncoder()
y_one_hot = encoder.fit_transform(y.reshape(-1, 1)).todense()

# Dividir em sets de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_spec, \
                                                    y_one_hot, \
                                                    test_size=0.25, \
                                                    random_state=42, \
                                                    stratify=y)

# Você pode normalizar os dados se quiser
# mas NÃO aplique redução de dimensionalidade nas imagens
...

In [ ]:
# Definição de rede CNN pronta para treinar
def build_model(input_size=(624, 129, 1), \
                n_classes=10, \
                conv_layers=(8, 16, 32), \
                ksize=5, \
                maxpool_size=3, \
                dense_layers=(32,), \
                dropout_rate=0.5, \
                conv_activation='relu', \
                dense_activation='sigmoid'):

    inputs = Input(shape=input_size)
    x      = inputs

    # Camadas convolucionais + camadas extras para melhorar o desempenho da rede
    for filters in conv_layers:
        x = Conv2D(filters=filters, kernel_size=ksize, padding='valid')(x)
        x = Activation(conv_activation)(x)
        x = MaxPool2D(pool_size=maxpool_size)(x)
        x = Dropout(rate=dropout_rate)(x)

    # Camadas densas
    x = Flatten()(x) # "Amassar" o array 2D em um array 1D
    for neurons in dense_layers:
        x = Dense(units=neurons, activation=dense_activation)(x)

    # Camada final, cada neurônio corresponde a uma classe
    outputs = Dense(units=n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, \
                  outputs=outputs, \
                  name='CNN_spectrograms_1')

    return model

In [ ]:
# Definir seed para resultados reproduzíveis
keras.utils.set_random_seed(42)

# Construir modelo com parâmetros default
model      = build_model()
learn_rate = 1e-3 # 0.001
opt        = Adam(learning_rate=learn_rate)
model.compile(loss = 'categorical_crossentropy', \
              optimizer = opt, \
              metrics = ['accuracy'])

In [ ]:
# Visualizar modelo - parte 1
model.summary()

In [ ]:
# Visualizar modelo - parte 2
visualkeras.layered_view(model, \
                         legend=True, \
                         draw_volume=True, \
                         scale_xy=5)

In [ ]:
# Treino
history = model.fit(X_train, \
                    y_train, \
                    validation_split=0.2, \
                    epochs=70, \
                    verbose=1)
# Partição de validação para acompanharmos o
# desempenho "real" da rede ao longo do treino

In [ ]:
# Plotar curvas de loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
# Plotar curvas de acurácia
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.legend()
plt.show()

In [ ]:
y_hat = model.predict(X_test)
y_hat = encoder.inverse_transform(y_hat).reshape(-1)
y_test2 = encoder.inverse_transform(np.asarray(y_test)).reshape(-1)

print(y_hat.shape)
print(y_test2.shape)

In [ ]:
# Avaliar o modelo
report_dict = classification_report(y_test2, y_hat, output_dict=True)

print(report_dict)